In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch

from bo_functions import ExactGPModel
from bo_functions import create_y_data
from bo_functions import bo_iter
from bo_functions import test_train_split
from bo_functions import bo_iter_w_runs
from bo_functions import find_train_doc_path
from datetime import datetime

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = None ##For Testing

#Set Parameters
Theta_True = np.array([1,-1])
BO_iters = 3
train_iter = 300
noise_std = 0.1
runs = 2
explore_bias = torch.tensor([0.75])
shuffle_seed = 9
set_lengthscale = np.array([None])
t= 100
emulator = True
sparse_grid = True
verbose = False
save_fig=True

#Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/n=5.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1]
Yexp = exp_data[:,2]
n = len(Xexp)

#Define GP Testing space
p=20
Theta1 =  np.linspace(-2,2,p) #1x10
Theta2 =  np.linspace(-2,2,p) #1x10
theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays

Date and Time:  28-Jul-2022 (16:54:22)


In [3]:
%%time
obj = "obj"

all_data_doc = find_train_doc_path(emulator, obj, t)
all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 

sep_fact = np.linspace(0.5,0.99,4)
# sep_fact = 0.8

print("Runs:", runs)
print("BO Iters:", BO_iters)
print("Number of Separation Factors Tested:", len(sep_fact))
print("----------------------------------------------------")
for i in range(len(sep_fact)):
    print("Separation Factor Train/Test:", str(np.round(sep_fact[i],3))
    print("Lengthscale Set To:", set_lengthscale)
    print("Initial Explore Bias:", str(np.round(float(explore_bias[0]),3)))
    results = bo_iter_w_runs(BO_iters,all_data_doc,t,theta_mesh,Theta_True,train_iter,explore_bias[0], Xexp, Yexp,
                                 noise_std, obj, runs, sparse_grid, emulator, set_lengthscale[0], verbose, 
                                 save_fig, shuffle_seed, DateTime, sep_fact = sep_fact[i])
    print("The GP predicts the lowest SSE of", "{:.3e}".format(np.exp(results[3])), "occurs at \u03B8 =", results[2][0], 
              "during run", results[1], "at BO iteration", results[0])
    print("At this point, the highest EI occurs at \u03B8 =", results[4][0])
    print("\n")

Runs: 2
BO Iters: 3
Number of Separation Factors Tested: 4
----------------------------------------------------
Separation Factor Train/Test: 0.5
Lengthscale Set To: [None]
Initial Explore Bias: 0.75


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


The GP predicts the lowest SSE of 6.285e-02 occurs at θ = [ 0.95206211 -1.03980998] during run 1 at BO iteration 3
At this point, the highest EI occurs at θ = [ 0.91377328 -1.02229358]


Separation Factor Train/Test: 0.6633333333333333
Lengthscale Set To: [None]
Initial Explore Bias: 0.75
The GP predicts the lowest SSE of 1.030e-01 occurs at θ = [ 0.91202051 -0.97091138] during run 1 at BO iteration 3
At this point, the highest EI occurs at θ = [ 0.9180631  -1.15780378]


Separation Factor Train/Test: 0.8266666666666667
Lengthscale Set To: [None]
Initial Explore Bias: 0.75
The GP predicts the lowest SSE of 1.034e-01 occurs at θ = [ 0.89252891 -1.00027806] during run 1 at BO iteration 3
At this point, the highest EI occurs at θ = [-1.78947368 -2.        ]


Separation Factor Train/Test: 0.99
Lengthscale Set To: [None]
Initial Explore Bias: 0.75
The GP predicts the lowest SSE of 2.550e-02 occurs at θ = [ 0.95768277 -0.98435236] during run 2 at BO iteration 3
At this point, the highest EI